In [40]:
import pandas as pd

full_df = pd.read_csv("Practical Example - Confidence Intervals.xlsx - Al Bundy.csv",skiprows=3, header=0)
full_df.set_index("InvoiceNo", inplace = True)

full_df = full_df.drop(columns = ["Unnamed: 0", "SalePrice", "Discount"])
full_df.rename(columns = {"Unnamed: 12" : "Year"}, inplace=True)

full_df

,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice
InvoiceNo,,,,,,,,,,,,,
52389,1/1/2014,United Kingdom,2152,UK2,Male,11.0,44,10.5,$ 159.00,0%,2014,1,$ 159.00
52390,1/1/2014,United States,2230,US15,Male,11.5,44-45,11.0,$ 199.00,20%,2014,1,$ 159.20
52391,1/1/2014,Canada,2160,CAN7,Male,9.5,42-43,9.0,$ 149.00,20%,2014,1,$ 119.20
52392,1/1/2014,United States,2234,US6,Female,9.5,40,7.5,$ 159.00,0%,2014,1,$ 159.00
52393,1/1/2014,United Kingdom,2222,UK4,Female,9.0,39-40,7.0,$ 159.00,0%,2014,1,$ 159.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65773,12/31/2016,United Kingdom,2154,UK2,Male,9.5,42-43,9.0,$ 139.00,0%,2016,12,$ 139.00
65774,12/31/2016,United States,2181,US12,Female,12.0,42-43,10.0,$ 149.00,0%,2016,12,$ 149.00
65775,12/31/2016,Canada,2203,CAN6,Male,10.5,43-44,10.0,$ 179.00,30%,2016,12,$ 125.30


In [24]:
men_df = full_df.loc[full_df["Gender"] == "Male"]
fem_df = full_df.loc[full_df["Gender"] == "Female"]
                     
m_canada = men_df["Country"] == "Canada"
m_us = men_df["Country"] == "United States"

men_canada_sales = men_df[m_canada]["Size (US)"].value_counts()
men_us_sales = men_df[m_us]["Size (US)"].value_counts()

men_df

,Unnamed: 0,Date,Country,ProductID,Shop,Gender,Size (US),Size (Europe),Size (UK),UnitPrice,Discount,Year,Month,SalePrice
InvoiceNo,,,,,,,,,,,,,,
52389,NaN,1/1/2014,United Kingdom,2152,UK2,Male,11.0,44,10.5,$ 159.00,0%,2014,1,$ 159.00
52390,NaN,1/1/2014,United States,2230,US15,Male,11.5,44-45,11.0,$ 199.00,20%,2014,1,$ 159.20
52391,NaN,1/1/2014,Canada,2160,CAN7,Male,9.5,42-43,9.0,$ 149.00,20%,2014,1,$ 119.20
52394,NaN,1/1/2014,United States,2173,US15,Male,10.5,43-44,10.0,$ 159.00,0%,2014,1,$ 159.00
52396,NaN,1/2/2014,Canada,2238,CAN5,Male,10.0,43,9.5,$ 169.00,0%,2014,1,$ 169.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65769,NaN,12/31/2016,Germany,2210,GER2,Male,10.5,43-44,10.0,$ 179.00,50%,2016,12,$ 89.50
65771,NaN,12/31/2016,United States,2209,US15,Male,9.0,42,8.5,$ 189.00,0%,2016,12,$ 189.00
65772,NaN,12/31/2016,United States,2168,US13,Male,8.0,41,7.5,$ 129.00,0%,2016,12,$ 129.00


In [25]:
men_df[m_canada]["Size (US)"].value_counts()

9.5     375
9.0     324
10.5    243
10.0    237
8.5     192
11.0    114
11.5     75
12.0     51
8.0      51
7.5      45
15.0     27
7.0      24
14.0     21
6.5      15
6.0      15
13.0     12
Name: Size (US), dtype: int64

In [38]:
filt_us = men_df.loc[(men_df["Country"] == "United States")]
sales = filt_us_16["Size (US)"].value_counts()

sales

#men_df.loc[filt_us_16].value_counts()

9.5     741
10.0    543
9.0     492
10.5    462
8.5     225
11.0    213
11.5    156
8.0     141
12.0     87
7.5      66
14.0     60
6.0      54
6.5      45
7.0      39
13.0     39
15.0     24
Name: Size (US), dtype: int64

In [26]:
men_df.dtypes

Unnamed: 0       float64
Date              object
Country           object
ProductID          int64
Shop              object
Gender            object
Size (US)        float64
Size (Europe)     object
Size (UK)        float64
UnitPrice         object
Discount          object
Year              object
Month              int64
SalePrice         object
dtype: object